# Lab 2 - TSP

The Traveling Salesman Problem asks about finding the shortest possible route that visits each cities (from a known list) exactly once and returns to the origin city. 

So we are looking for a minimum cost hamiltonian cycle, inside a fully connected graph.

In [1]:
## Import

import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [5]:
## Loading cities from the directory

CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])

## Initializing the distance matrix -> distance between couple of cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
    
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## First solution 